In [1]:
import numpy as np
import pandas as pd 
import torch,os
from torchvision import datasets ,transforms
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader,Subset
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import models
from torch import optim
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [2]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master/')
import timm
from timm import create_model

In [3]:
train_data=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_data=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [4]:
class ImageDataset(Dataset):    
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_labels = train_data[['Id','Pawpularity']].copy()
        self.img_dir = img_dir 
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path+'.jpg')
        image_labels = self.img_labels.iloc[idx,1]
        if self.transform:
            image = self.transform(image)
        return image,image_labels

In [5]:
#dataset for testing
class ImageDataset_test(Dataset):
    
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_id = test_data[['Id']].copy()
        self.img_dir = img_dir 
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_id)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_id.iloc[idx, 0])
        image = read_image(img_path+'.jpg')
        image_id = self.img_id.iloc[idx,0]
        if self.transform:
            image = self.transform(image)
        return image,image_id

In [6]:


transformed_dataset =  ImageDataset(img_dir='../input/petfinder-pawpularity-score/train/',transform=transforms.Compose([
    transforms.Resize(256),transforms.CenterCrop(224),transforms.RandomHorizontalFlip(),
     transforms.ConvertImageDtype(torch.float), transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])]))



In [7]:
train_features=['Subject Focus','Eyes','Face','Near','Action','Accessory','Group','Collage','Human','Occlusion','Info','Blur']


temp_data=train_data[train_features]

In [8]:


kf=  StratifiedKFold(n_splits=5,random_state=42,shuffle=True)



In [9]:


def model_initialise():
    model=create_model('swin_large_patch4_window7_224',pretrained=True)
    model.head=nn.Sequential(nn.Linear(1536,64),nn.BatchNorm1d(64),nn.ReLU(),nn.Linear(64,1))
    for param in model.parameters():
        param.require_grad=False
    for param in model.head.parameters():
        param.require_grad=True
    model.to('cuda:0')
    return model



In [10]:
def train_model(fold,model,train_data_loader,val_data_loader):
    criterion=nn.MSELoss()
    optimizer=optim.AdamW(model.head.parameters(),weight_decay=5e-3,lr=1e-3)
    reduce_lr= optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.6,verbose=True)
    #--------------------------------------------------------------------------------------------------
    best_model=model
    best_val=1000
    num_epochs=5
    
#-------------------------------------------------------------------------------------------
    running_loss=0.0
    print('---------------------Training model now.......')
    for epoch in range(num_epochs):
        model.train()
        running_loss=0.0
        for i,(train_features,train_labels) in enumerate(train_data_loader):
            train_features=train_features.to('cuda:0')
            train_labels=train_labels.to('cuda:0')
            train_labels=train_labels.to(torch.float32)
            #rint(plt.hist((train_labels.cpu().numpy())))
            train_labels=train_labels.view(-1,1)
            #train_labels/=100.0
            out=model(train_features)
            #print(out[0],train_labels[0])
            loss = criterion(out, train_labels)
            with torch.no_grad():
                running_loss+=np.sqrt(loss.item()) 
            loss.backward()  
            
            
        # Update model parameters
            optimizer.step()
            optimizer.zero_grad()
            del out,loss,train_features,train_labels
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
    
    
        with torch.no_grad():
            print('fold:',fold,' avg loss after ',epoch+1,' epochs:',running_loss/233)
            val_loss_total=0
            model.eval()
            for i,(val_features,val_labels) in enumerate(val_data_loader):
                val_features,val_labels=next(iter(val_data_loader))
                val_features=val_features.to('cuda:0')
                val_labels=val_labels.to('cuda:0')
                val_labels=val_labels.to(torch.float32)
                #val_labels/=100
                val_out=model(val_features)
                val_labels=val_labels.view(-1,1)
                val_loss=criterion(val_out,val_labels)
                val_loss_total+=np.sqrt(val_loss.item())
                del val_out,val_loss,val_features,val_labels
            
                torch.cuda.empty_cache()
                torch.cuda.synchronize()
            reduce_lr.step()
            print('validation loss after epoch ',epoch+1,':',val_loss_total/78)
            if(val_loss_total/78<best_val):
                best_val=val_loss_total/78
                best_model=model
                
    #finetune(fold,model,train_data_loader,val_data_loader)
    save_model(best_model,best_val,fold)

In [11]:


def save_model(best_model,best_val,fold):
    torch.save(best_model.state_dict(),'model_'+str(fold)+'_weights.pth')
    print('----------Fold:',fold,' ,model saved with val_accuracy:',best_val,'------------------')



In [12]:


fold=0
for train_index, test_index in  kf.split(np.zeros(temp_data.shape),train_data['Pawpularity']):
    fold=fold+1
    
    train_split=Subset(transformed_dataset,train_index)
    test_split=Subset(transformed_dataset,test_index)
    #print(train_split)
    #print('test split:',test_split)
    train_data_loader=DataLoader(train_split,batch_size=32,shuffle=True,num_workers=2,pin_memory=True)
    val_data_loader=DataLoader(test_split,batch_size=32,shuffle=False,num_workers=2,pin_memory=True)
    #print(next(iter(train_data_loader)))
    #print(next(iter(val_data_loader)))
    model=model_initialise() 
    train_model(fold,model,train_data_loader,val_data_loader)
 


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window7_224_22kto1k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth


Adjusting learning rate of group 0 to 1.0000e-03.
---------------------Training model now.......
fold: 1  avg loss after  1  epochs: 40.6063087346095
Adjusting learning rate of group 0 to 1.0000e-03.
validation loss after epoch  1 : 23.757585181872425
fold: 1  avg loss after  2  epochs: 26.177763665800942
Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  2 : 12.096174045224556
fold: 1  avg loss after  3  epochs: 19.05640798048966
Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  3 : 10.666304976362834
fold: 1  avg loss after  4  epochs: 17.95339998869538
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  4 : 10.519496929143529
fold: 1  avg loss after  5  epochs: 17.54938934133827
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  5 : 10.391031507199228
----------Fold: 1  ,model saved with val_accuracy: 10.391031507199228 ------------------
Adjusting learning rate of group 0

Traceback (most recent call last):


Adjusting learning rate of group 0 to 1.0000e-03.
validation loss after epoch  1 : 27.527192846972003
fold: 2  avg loss after  2  epochs: 26.124576370042153
Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  2 : 19.243773394119042
fold: 2  avg loss after  3  epochs: 19.044264874019866
Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  3 : 18.188273095265856
fold: 2  avg loss after  4  epochs: 17.96152474342631
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  4 : 18.201695539601904
fold: 2  avg loss after  5  epochs: 17.500290627038385
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  5 : 18.130855083116554
----------Fold: 2  ,model saved with val_accuracy: 18.130855083116554 ------------------
Adjusting learning rate of group 0 to 1.0000e-03.
---------------------Training model now.......
fold: 3  avg loss after  1  epochs: 40.179250453813665
Adjusting learning rate of gro

Traceback (most recent call last):


Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  3 : 16.155027811622038
fold: 3  avg loss after  4  epochs: 18.008651652725934
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  4 : 16.58107288669583
fold: 3  avg loss after  5  epochs: 17.532271631179448


Traceback (most recent call last):


Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  5 : 16.062098076994932
----------Fold: 3  ,model saved with val_accuracy: 16.062098076994932 ------------------
Adjusting learning rate of group 0 to 1.0000e-03.
---------------------Training model now.......
fold: 4  avg loss after  1  epochs: 40.37917540994289
Adjusting learning rate of group 0 to 1.0000e-03.
validation loss after epoch  1 : 23.3766331076199
fold: 4  avg loss after  2  epochs: 26.39133122232366
Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  2 : 13.199782647546714
fold: 4  avg loss after  3  epochs: 19.21905076752003
Adjusting learning rate of group 0 to 6.0000e-04.
validation loss after epoch  3 : 12.01169831595139
fold: 4  avg loss after  4  epochs: 18.112903411353425
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  4 : 11.106774434781906
fold: 4  avg loss after  5  epochs: 17.718044699194813
Adjusting learning rate of group 0 

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)


Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  4 : 14.550580429676844
fold: 5  avg loss after  5  epochs: 17.654385935367106
Adjusting learning rate of group 0 to 3.6000e-04.
validation loss after epoch  5 : 14.30881192051414
----------Fold: 5  ,model saved with val_accuracy: 14.30881192051414 ------------------
